In [400]:
%config InlineBackend.figure_format = "retina"

In [401]:
import os
import sys
import re

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import talib
import pandas_ta as pta

import time
import datetime

In [402]:
class CDXData:
    def __init__(self, starttime, endtime, symbol, interval, exchange_code):
        self.start = starttime  # "DD-MM-YYYY
        self.end = endtime
        self.interval = interval
        self.symbol = symbol
        self.exchange_code = exchange_code

    def convert_to_millisecond(self, time):
        date, month, year = time.split(" ")[0].split("-")
        hours, minutes, seconds = time.split(" ")[1].split(":")
        date_time = datetime.datetime(int(year), int(
            month), int(date), int(hours), int(minutes))

        return datetime.datetime.timestamp(date_time)*1000

    def fetch_api_call(self, start, end):

        url = f"https://public.coindcx.com/market_data/candles?pair={self.exchange_code.upper()}-{self.symbol.upper()}&interval={self.interval}&startTime={start}&endTime={end}&limit=1000" # Replace 'SNTBTC' with the desired market pair.

        response = requests.get(url)
        data = response.json()
        return data

    def fetch_range_data(self):
        date, month, year = self.start.split("-")
        start_dt = datetime.date(int(year), int(month), int(date))
        date, month, year = self.end.split("-")
        end_dt = datetime.date(int(year), int(month), int(date))

        delta = datetime.timedelta(days=1)

        dates = []

        while start_dt <= end_dt:
            t1 = self.convert_to_millisecond(
                str(start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+" 00:00:00")
            t2 = self.convert_to_millisecond(
                str(start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+" 23:59:59")
            data = self.fetch_api_call(t1, t2)
            df = pd.DataFrame(data)

            while len(data) == 1000:
                t2 = df["time"].iloc[-1]
                data = self.fetch_api_call(t1, t2)
                df = pd.concat([df, pd.DataFrame(data)])

            df.to_csv("cdx_bin_btcusdt/CDX_"+self.symbol+"_"+str(start_dt.day) +
                      "-"+str(start_dt.month)+"-"+str(start_dt.year)+".csv", index=False)
            start_dt += delta
        print(dates)

    def get_all_data(self):
        date, month, year = self.start.split("-")
        start_dt = datetime.date(int(year), int(month), int(date))
        date, month, year = self.end.split("-")
        end_dt = datetime.date(int(year), int(month), int(date))

        delta = datetime.timedelta(days=1)
        df = pd.DataFrame()
        while start_dt <= end_dt:
            df_new = pd.read_csv("cdx_bin_btcusdt/CDX_"+self.symbol+"_"+str(
                start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+".csv")
            start_dt += delta
            df = pd.concat([df, df_new], axis=0)

        # sort df on basis of time ascending
        df = df.sort_values(by=['time'])
        df.to_csv("CDX_final.csv", index=False)
        return df


In [403]:
def get_sma(df):
    plt.figure(figsize=(20, 10))

    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    y = df["open"]
    
    window_size = 10
    # Calculate the simple moving average using the rolling() and mean() functions
    sma = df["open"].rolling(window=window_size).mean()
    df["sma_10"] = sma

In [404]:
def get_sma_cross(t1, t2, df):
    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    y = df["open"]

    plt.figure(figsize=(20, 10))
    
    sma_low = df["open"].rolling(window=t1).mean()
    sma_high = df["open"].rolling(window=t2).mean()
    df["sma_low"] = sma_low
    df["sma_high"] = sma_high

    # write to csv
    df.to_csv("CDX_sma.csv", index=False)

In [405]:
def get_bbands(df):
    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    upper, middle, lower = talib.BBANDS(df["close"], timeperiod=20)
    bbands_talib = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                      "bb_ma": middle,
                                      "bb_high": upper}
                                )

    df["bb_low"] = bbands_talib["bb_low"]
    df["bb_ma"] = bbands_talib["bb_ma"]
    df["bb_high"] = bbands_talib["bb_high"]


In [406]:
def get_tick(ind, df): 
    return df.iloc[ind]

In [407]:
def apply_sma_cross(t1, t2, df):
    temp = df.iloc[:t2]
    temp_df = pd.DataFrame(temp)
    
    # create a dataframe with 2 columns sma_low and sma_high
    sma_df = pd.DataFrame(columns=["sma_low", "sma_high"])
    initial_row = pd.DataFrame({"sma_low": [0], "sma_high": [0]})
    sma_df = pd.concat([sma_df, initial_row], ignore_index=True)

    fig, ax = plt.subplots()
    line, = ax.plot(sma_df["sma_low"], sma_df["sma_high"], color="red")
    

    for i in range(t2, len(df)):
        # get the tick
        tick = get_tick(i, df)
        temp_df = pd.concat([temp_df, tick], ignore_index=True)

        sma_low_prev = sma_df["sma_low"].iloc[-1]
        sma_high_prev = sma_df["sma_high"].iloc[-1]

        sma_low_new = (sma_low_prev*t1 - temp_df.iloc[-t1, -6] + tick.iloc[-6])/t1
        sma_high_new = (sma_high_prev*t2 - temp_df.iloc[-t2, -6] + tick.iloc[-6])/t2

        new_row = pd.DataFrame({"sma_low": [sma_low_new], "sma_high": [sma_high_new]})
        sma_df = pd.concat([sma_df, new_row], ignore_index=True)

        line.set_data(sma_df["sma_low"], sma_df["sma_high"])

         # Adjust the x-axis limits if needed
        ax.relim()
        ax.autoscale_view()

        # Redraw the plot
        plt.draw()
        plt.pause(0.1)

        

    plt.show()    

    # update the sma_low and sma_high of the tick
    # temp_df.loc[t2, "sma_low"] = sma_low
    # temp_df.loc[t2, "sma_high"] = sma_high

    

In [408]:
def apply_sma_cross_new(t1, t2, df):
    assert(len(df) > t2)
    temp = df.iloc[:t2]
    temp_df = pd.DataFrame(temp)
    
    # # create a dataframe with 2 columns sma_low and sma_high
    time = temp_df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    sma_low, sma_high = [], []


    # # fig, ax = plt.subplots()
    # # line, = ax.plot(time, sma_low, color="red")
    
    sma_low = temp_df["open"].rolling(window=t1).mean()
    sma_high = temp_df["open"].rolling(window=t2).mean()
    sma_df = pd.DataFrame({"sma_low": sma_low, "sma_high": sma_high})

    for i in range(0, len(df)):

        tick = get_tick(t2, df)

        # print in float
        print("______________________Printing old tempdf_______________________/n")
        print(temp_df)
        print("--------------------old tempdf over------------------------/n")

        print("__________________Printing tick__________________/n")
        print(tick)
        print("______________________Tick over_____________________/n")
        temp_df = pd.concat([temp_df, pd.DataFrame({'open': tick["open"], 'high': tick["high"], 'low': tick["low"], 'volume': tick["volume"],'close': tick["close"], 'time': tick["time"]}, index=[0])], axis=0)
        print("____________________new temp df_______________________/n")
        print(temp_df)
        print("____________________new temp df over________________________/n")

        sma_low_prev = sma_df.iloc[-1, 0]
        sma_high_prev = sma_df.iloc[-1, 1]


        # print(sma_low_prev, sma_high_prev)

        sma_low_new = (sma_low_prev*t1 - temp_df.iloc[-t1, -6] + tick.iloc[-6])/t1
        sma_high_new = (sma_high_prev*t2 - temp_df.iloc[-t2, -6] + tick.iloc[-6])/t2


        new_row = pd.DataFrame({"sma_low": [sma_low_new], "sma_high": [sma_high_new]})

        # print(sma_df)
        # print("old one over_______________________________________")
        sma_df = pd.concat([sma_df, new_row], ignore_index=True)
        # print(sma_df)

        # print("--------------------------------------------/n")

        # if(sma_low_new == sma_high_new):
        #     print("Critical point")
       

        # line.set_data(sma_low, sma_high)

        #  # Adjust the x-axis limits if needed
        # ax.relim()
        # ax.autoscale_view()

        # # Redraw the plot
        # plt.draw()   
    # sma_df = pd.DataFrame({"sma_low": sma_low, "sma_high": sma_high})
    # sma_df.to_csv("CDX_sma_cross.csv", index=False)
    # update the sma_low and sma_high of the tick
    # temp_df.loc[t2, "sma_low"] = sma_low
    # temp_df.loc[t2, "sma_high"] = sma_high


In [409]:
obj = CDXData("31-12-2022", "31-12-2022", "BTC_USDT", "1h", "B")
# obj.fetch_range_data()
df = obj.get_all_data()
apply_sma_cross_new(5, 10, df)
df = get_sma_cross(5, 10, df)

df
# get_sma_cross(20, 40)
# get_bbands(df)




# time = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
# price, sma_low, sma_high, bb_low, bb_ma, bb_high = df["open"], df["sma_low"], df["sma_high"], df["bb_low"], df["bb_ma"], df["bb_high"]

# plt.plot(time, price, label='Price', color='blue')
# plt.plot(time, sma_low, label='SMA_Low', color='orange')
# plt.plot(time, sma_high, label='SMA_High', color='green')

# df

# plt.plot(time, bb_low, color="red")
# plt.plot(time, bb_ma, color="blue")
# plt.plot(time, bb_high, color="red")

______________________Printing old tempdf_______________________/n
        open      high       low        volume     close           time
23  16565.77  16570.53  16523.81  402581.80979  16525.37  1672426800000
22  16525.91  16571.51  16523.91  398842.67338  16568.69  1672430400000
21  16569.36  16677.35  16568.31  881213.09187  16581.88  1672434000000
20  16585.06  16597.00  16577.57  264471.11964  16583.27  1672437600000
19  16582.81  16618.75  16568.00  407840.29825  16607.48  1672441200000
18  16607.48  16616.37  16578.00  318703.77918  16580.32  1672444800000
17  16580.32  16583.69  16562.36  307357.87124  16568.24  1672448400000
16  16568.23  16579.79  16560.47  270129.94923  16575.94  1672452000000
15  16576.18  16577.38  16550.01  263425.81138  16552.46  1672455600000
14  16552.46  16555.59  16541.75  206382.72605  16550.99  1672459200000
--------------------old tempdf over------------------------/n
__________________Printing tick__________________/n
open      1.655189e+04
high

<Figure size 2000x1000 with 0 Axes>